## Parsing REPBULIC hOCR Files

The scans of the printed RSG volumes have the following characteristics

- all scans:
  - have two pages per scan
  - have up to 4 columns per scan, 2 per page 
  - full scan is around 4800 pixels wide, left page is up to pixel 2400, right page is from pixel 2400 (roughly)
- scans of index pages
  - have no page numbers
- scans of resolution pages
  - have page numbers (left-side page is even, right-side page is odd)
  
### Columns

The scans are normalized such that the columns are straight. The text width should be around 1000 pixels. Some columns are not cut out properly, resulting in columns that are either to small (some of the column text is missing), or too wide (the hOCR output contains partial texts from two columns)

### Index pages

- start of entry: 
  - start left alignment
- end of entry:
  - end of line possibly before end of text column. 
  - One or more page numbers


### Resolution pages

- header:
  - next top of page (less than 350 pixels from the top)
  - page has header with:
    - even numbered pages: date page_number year
    - odd numbered pages: year page_number date
  - columns have half of page header, e.g.:
    - even numbered pages: 
      - first column: date left aligned and part of page_number right aligned
      - second column: part of page_number left aligned and year right aligned
    - odd numbered pages: 
      - first column: year left aligned and part of page_number right aligned
      - second column: part of page_number left aligned and date right aligned
      
### Viewer

- page viewer: https://images.huygens.knaw.nl/assets/argos/index.html
- list of page URLs: https://images.huygens.knaw.nl/api/argos


### National Archive site

- search in the archive: https://www.nationaalarchief.nl/onderzoeken/index/nt00444?searchTerm=
- search the index: https://www.nationaalarchief.nl/onderzoeken/zoekhulpen/voc-opvarenden
- example page: https://www.nationaalarchief.nl/onderzoeken/index/nt00444/d110980c-c864-11e6-9d8b-00505693001d


In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2

In [7]:
import json
import os
import re
from collections import defaultdict
from parse_hocr_files import make_hocr_page
#from parse_republic_hocr_files import get_files, get_page_types, count_page_ref_lines, get_index_entry_lines, gather_page_columns
from elasticsearch import Elasticsearch
import parse_republic_hocr_files as rep_parser

# The hOCR file name contains relevant information for parsing. Here's an example:
# NL-HaNA_1.01.02_3780_0016.jpg-0-251-98--0.40.hocr

# NL-HaNA_1.01.02 is the name of the archive
# 3780_0016 identifies the specific page with a specific contract
# 0-251-98--0.40 identifies four aspects:
#   1. the number of the column (0)
#   2. the offset from the left (251)
#   3. the offset from the top (98)
#   4. and the slant (-0.40)



### Reading column scans for a single volume

1. get scan file info
    - scan number, page number, page side, column number, slant, page
2. iterate over pages
    - create hocr_page
    - determine page type: index, resolution, other
    

In [346]:
import copy

data_dir = "../../../Data/Projects/REPUBLIC/hocr/1725/"

scan_files = rep_parser.get_files(data_dir)
scan_files.sort(key = lambda x: x["scan_num_column_num"])
print("Number of scan files:", len(scan_files))
#print(json.dumps(scan_files[0:11], indent=2))

scan_columns = defaultdict(list)
for scan_file in scan_files:
    scan_columns[scan_file["scan_num"]]

config = {
    "tiny_word_width": 15, # pixel width
    "avg_char_width": 20,
    "remove_tiny_words": True,
    "remove_line_numbers": False,
}

pages_info = rep_parser.gather_page_columns(scan_files)


Number of scan files: 1887


### Indexing Page Data in Elasticsearch



In [201]:
index = "republic_hocr_pages"
doc_type = "page"

es = Elasticsearch()
if es.indices.exists(index=index):
    print("exists, deleting")
    es.indices.delete(index=index)

def create_es_doc(page_info):
    doc = copy.deepcopy(page_info)
    for column_info in doc["columns"]:
        if "column_hocr" in column_info:
            column_info["column_hocr"] = json.dumps(column_info["column_hocr"])
    return doc

def parse_es_doc(doc):
    for column_info in doc["columns"]:
        if "column_hocr" in column_info:
            column_info["column_hocr"] = json.loads(column_info["column_hocr"])
    return doc

for page_id in pages_info:
    if pages_info[page_id]["scan_num"] >= 7000:
        continue
    print(page_id)
    page_info = copy.deepcopy(pages_info[page_id])
    for column_info in page_info["columns"]:
        try:
            column_info["column_hocr"] = rep_parser.get_column_hocr(column_info, config)
        except TypeError:
            print("Error parsing file", column_info["filepath"])
    doc = create_es_doc(page_info)
    es.index(index=index, doc_type=doc_type, id=page_id, body=doc)
        
    #pages_hocr[page_id] = rep_parser.get_page_columns_hocr(pages_info[page_id], config)
#print(json.dumps(pages_info["scan-6-even"], indent=2))
#print(json.dumps(pages_hocr["scan-6-even"], indent=2))


exists, deleting
scan-4-even
scan-4-odd
scan-5-even
scan-5-odd
scan-6-even
scan-6-odd
scan-7-even
scan-7-odd
scan-8-even
scan-8-odd
scan-9-even
scan-9-odd
scan-10-even
scan-10-odd
scan-11-even
scan-11-odd
scan-12-even
scan-12-odd
scan-13-even
scan-13-odd
scan-14-even
scan-14-odd
scan-15-even
scan-15-odd
scan-16-even
scan-16-odd
scan-17-even
scan-17-odd
scan-18-even
scan-18-odd
scan-19-even
scan-19-odd
scan-20-even
scan-20-odd
scan-21-even
scan-21-odd
scan-22-even
scan-22-odd
scan-23-even
scan-23-odd
scan-24-even
scan-24-odd
scan-25-even
scan-25-odd
scan-26-even
scan-26-odd
scan-27-even
scan-27-odd
scan-28-even
scan-28-odd
scan-29-even
scan-29-odd
scan-30-even
scan-30-odd
scan-31-even
scan-31-odd
scan-32-even
scan-32-odd
scan-33-even
scan-33-odd
scan-34-even
scan-34-odd
scan-35-even
scan-35-odd
scan-36-even
scan-36-odd
scan-37-even
scan-37-odd
scan-38-even
scan-38-odd
scan-39-even
scan-39-odd
scan-40-even
scan-40-odd
scan-41-even
scan-41-odd
scan-42-even
scan-42-odd
scan-43-even
scan-43

scan-322-odd
scan-323-even
scan-323-odd
scan-324-even
scan-324-odd
scan-325-odd
scan-326-even
scan-326-odd
scan-327-even
scan-327-odd
scan-328-even
scan-328-odd
scan-329-even
scan-329-odd
scan-330-even
scan-330-odd
scan-331-even
scan-331-odd
scan-332-even
scan-332-odd
scan-333-even
scan-333-odd
scan-334-even
Error parsing file ../../../Data/Projects/REPUBLIC/hocr/1725/NL-HaNA_1.01.02_3780_0334.jpg-2-2253-75--0.00.hocr
scan-334-odd
scan-335-even
scan-335-odd
scan-336-even
scan-336-odd
scan-337-even
scan-337-odd
scan-338-even
scan-338-odd
scan-339-even
scan-339-odd
scan-340-even
scan-340-odd
scan-341-even
scan-341-odd
scan-342-even
scan-342-odd
scan-343-even
scan-343-odd
scan-344-even
scan-344-odd
scan-345-even
scan-345-odd
scan-346-even
scan-346-odd
scan-347-even
scan-347-odd
scan-348-even
scan-348-odd
scan-349-even
scan-350-even
scan-350-odd
scan-351-even
scan-351-odd
scan-352-even
scan-352-odd
scan-353-even
scan-353-odd
scan-354-even
scan-354-odd
scan-355-even
scan-355-odd
scan-356-ev

### Determining Page Type

We want to parse index pages differently from resolution pages and filter out non-text pages and pages where the columns are not properly identified.


In [404]:
def retrieve_page_doc(page_id):
    response = es.get(index=index, doc_type=doc_type, id=page_id)
    if "_source" in response:
        page_doc = parse_es_doc(response["_source"])
        return page_doc
    else:
        return None
    

for page_id in pages_info:
    #if pages_info[page_id]["scan_num"] < 14 or  pages_info[page_id]["scan_num"] > 318:
    #    continue
    page_doc = retrieve_page_doc(page_id)
    #print(json.dumps(page_info, indent=2))
    #print(pages_hocr[page_id]['scan-4-even-0'])
    rep_parser.calculate_left_jumps(page_doc)
    pages_info[page_id]["page_type"] = rep_parser.get_page_type(page_doc, config, DEBUG=False)
    print(page_id, "\t", pages_info[page_id]["page_type"])




scan-4-even 	 bad_page
scan-4-odd 	 bad_page
		NO HEADER LINE for column id scan-5-even-0
scan-5-even 	 index_page
scan-5-odd 	 index_page
scan-6-even 	 bad_page
scan-6-odd 	 index_page
scan-7-even 	 index_page
scan-7-odd 	 index_page
scan-8-even 	 index_page
scan-8-odd 	 index_page
scan-9-even 	 bad_page
scan-9-odd 	 index_page
scan-10-even 	 index_page
scan-10-odd 	 index_page
scan-11-even 	 bad_page
scan-11-odd 	 index_page
scan-12-even 	 index_page
scan-12-odd 	 index_page
scan-13-even 	 index_page
scan-13-odd 	 index_page
scan-14-even 	 index_page
scan-14-odd 	 bad_page
		NO HEADER LINE for column id scan-15-even-1
scan-15-even 	 index_page
scan-15-odd 	 bad_page
scan-16-even 	 index_page
scan-16-odd 	 index_page
scan-17-even 	 index_page
scan-17-odd 	 index_page
scan-18-even 	 index_page
scan-18-odd 	 bad_page
scan-19-even 	 index_page
scan-19-odd 	 bad_page
scan-20-even 	 index_page
scan-20-odd 	 bad_page
scan-21-even 	 index_page
scan-21-odd 	 bad_page
scan-22-even 	 index_page

scan-133-even 	 resolution_page
scan-133-odd 	 resolution_page
scan-134-even 	 resolution_page
scan-134-odd 	 resolution_page
scan-135-even 	 resolution_page
scan-135-odd 	 resolution_page
scan-136-even 	 resolution_page
scan-136-odd 	 resolution_page
scan-137-even 	 resolution_page
scan-137-odd 	 resolution_page
		NO HEADER LINE for column id scan-138-even-0
scan-138-even 	 resolution_page
scan-138-odd 	 resolution_page
scan-139-even 	 resolution_page
scan-139-odd 	 resolution_page
scan-140-even 	 resolution_page
scan-140-odd 	 resolution_page
scan-141-even 	 resolution_page
scan-141-odd 	 resolution_page
scan-142-even 	 resolution_page
scan-142-odd 	 resolution_page
scan-143-even 	 resolution_page
scan-143-odd 	 resolution_page
scan-144-even 	 resolution_page
scan-144-odd 	 resolution_page
scan-145-odd 	 resolution_page
scan-146-even 	 resolution_page
scan-146-odd 	 resolution_page
scan-147-even 	 resolution_page
scan-147-odd 	 resolution_page
scan-148-even 	 resolution_page
scan-148

scan-266-even 	 resolution_page
		NO HEADER LINE for column id scan-266-odd-2
scan-266-odd 	 resolution_page
scan-267-even 	 resolution_page
scan-267-odd 	 resolution_page
scan-268-even 	 resolution_page
scan-268-odd 	 resolution_page
scan-270-even 	 resolution_page
scan-270-odd 	 resolution_page
scan-271-even 	 resolution_page
scan-271-odd 	 resolution_page
scan-272-even 	 resolution_page
scan-272-odd 	 resolution_page
scan-273-even 	 resolution_page
scan-273-odd 	 resolution_page
scan-274-even 	 resolution_page
scan-274-odd 	 resolution_page
scan-275-even 	 resolution_page
scan-275-odd 	 resolution_page
scan-276-even 	 resolution_page
scan-276-odd 	 resolution_page
scan-277-even 	 resolution_page
scan-277-odd 	 resolution_page
scan-278-even 	 resolution_page
scan-278-odd 	 resolution_page
scan-279-even 	 resolution_page
scan-279-odd 	 resolution_page
scan-280-even 	 resolution_page
scan-280-odd 	 resolution_page
scan-281-even 	 resolution_page
scan-281-odd 	 resolution_page
scan-282-

scan-402-odd 	 resolution_page
scan-403-even 	 resolution_page
scan-403-odd 	 resolution_page
scan-404-even 	 resolution_page
scan-404-odd 	 resolution_page
scan-405-even 	 resolution_page
scan-405-odd 	 resolution_page
scan-406-even 	 resolution_page
scan-406-odd 	 resolution_page
scan-407-even 	 resolution_page
scan-407-odd 	 resolution_page
scan-408-even 	 resolution_page
scan-408-odd 	 resolution_page
scan-409-even 	 resolution_page
scan-409-odd 	 resolution_page
scan-410-even 	 resolution_page
scan-410-odd 	 resolution_page
scan-411-even 	 resolution_page
scan-411-odd 	 resolution_page
scan-412-even 	 resolution_page
scan-412-odd 	 resolution_page
scan-413-even 	 resolution_page
scan-413-odd 	 resolution_page
scan-414-even 	 resolution_page
scan-414-odd 	 resolution_page
scan-415-even 	 resolution_page
scan-415-odd 	 resolution_page
		NO HEADER LINE for column id scan-416-even-0
scan-416-even 	 resolution_page
scan-416-odd 	 resolution_page
scan-417-even 	 resolution_page
scan-417

### Parsing and Preprocessing Index Pages

- filter tiny and huge text elements (i.e. deviating from average character/word width and height
- extract page lines that are part of the main text body containing index entries
- insert and clean up repetition symbols in index entries
    - determine length of repetition symbol
    - identify and replace mis-recognized repetition symbols


In [341]:
from parse_republic_hocr_files import index_lemmata
avg_left = 0
lemma_index = defaultdict(list)
curr_lemma = None
    

for page_id in pages_info:
    print("\n\n", page_id)
    if pages_info[page_id]["page_type"] != "index_page":
        print("skipping non-index page")
        continue
    page_doc = retrieve_page_doc(page_id)
    pages_info[page_id]["num_page_ref_lines"] = rep_parser.count_page_ref_lines(page_doc)
    for column_info in page_doc["columns"]:
        print("\n\n", column_info["column_id"])
        column_hocr = column_info["column_hocr"]
        lines = rep_parser.get_index_entry_lines(column_hocr)
        curr_lemma = index_lemmata(column_id, lines, lemma_index, curr_lemma)






 scan-4-even
skipping non-index page


 scan-4-odd
skipping non-index page


 scan-5-even
skipping non-index page


 scan-5-odd


 scan-5-odd-1
Header: LE        Kn
	scan: don't know line: 0 top: 244 0 1004 1004
	FIRST BODY LINE: 9 1903 	##            gejaf} Fabritius te'Cadix eh Courcamp
possibly misrecognised repeat symbol: 145 mn
DEVIATING LINE: 227 [227, 108, 63, 112, 169] gejaf} Fabritius te'Cadix eh Courcamp
avg_repeat_symbol_length: 145
DEVIATING LINE: 169 [227, 108, 63, 112, 169, 114, 116, 113, 113] ee juftificatie van de conduites ‘gehouden
avg_repeat_symbol_length: 145
DEVIATING LINE: 237 [114, 116, 113, 113, 237, 110, 114, 60, 116] redenen waarom het Schip van Zee-
avg_repeat_symbol_length: 145
DEVIATING LINE: 240 [114, 60, 116, 108, 240, 69, 115, 72, 119] gevis en geaccordeert. “916.
avg_repeat_symbol_length: 145
DEVIATING LINE: 247 [115, 72, 119, 122, 247, 122, 260, 124, 131] Re{olutie ‘wegens de adminiftratie van
avg_repeat_symbol_length: 145
DEVIATING LINE: 260 [119, 1

41 101 1 continue_stop 	      de Convoyen en Licenten tot Swolle.   169.
42 99 -2 start 	      —— wegens  aangevinge  van Rundvee én
	PAGE_REFS: [178] 	CURR LEMMA: Admiraliteyt in het Noorder Quartier
HAS LEMMA: Varckens. 178.
LEMMA: Varckens
43 100 -1 start_stop 	      Varckens.   178.
44 100 -2 start 	      —— Greveftein  aangefteldt  tot  Contrerol-
45 108 6 continue 	      leur van de Comvoyen en Licenten tot Ha(fel.
	PAGE_REFS: [256] 	CURR LEMMA: Varckens
46 107 5 continue_stop 	      256.
47 100 -3 start 	      —— commijfie  wegens  Hollandt  voor  de
	PAGE_REFS: [304] 	CURR LEMMA: Varckens
48 106 10 continue_stop 	      Heer Jelius.   304.
49 98 6 continue 	      —— iten voor de Heer Aylua wegens Pries-
	PAGE_REFS: [310] 	CURR LEMMA: Varckens
50 106 14 continue_stop 	      landt.   310.
51 108 21 continue 	 —— Akerlaken  aangefteldt  zot  Capiteyn,
52 37 -52 start 	   uw  35
53 59 -33 start 	    and    Houttuyn aangefteldt  tot Capiteya op
	PAGE_REFS: [361] 	CURR LEMMA: Varckens

5 104 -15 start 	    —— Ei   en Zoelmont   midts vier duy[ent
	PAGE_REFS: [166] 	CURR LEMMA: Beaufort aangefteldt tot Secretaris
6 150 37 continue_stop 	        gulden ter The[aurie fourneerende.   166.
HAS LEMMA: Becker, creditif van den Furft van Ooftvries-
LEMMA: Becker
7 104 -8 start 	      Becker, creditif van den Furft van Ooftvries-
	PAGE_REFS: [31] 	CURR LEMMA: Becker
8 49 -56 start_stop 	        —— landt.   31.
	PAGE_REFS: [96] 	CURR LEMMA: Becker
9 101 7 continue_stop 	      —— creditif en aangenaam.  96.
10 105 11 continue 	      —— Memorie tegens  de afgefette  admini-
	PAGE_REFS: [126] 	CURR LEMMA: Becker
11 141 53 continue_stop 	        fratoiren, te examineeren.   126.
12 49 -38 start 	   —— rapport  en  refolutie  dien aangaande.
	PAGE_REFS: [137] 	CURR LEMMA: Becker
13 49 -44 start_stop 	         —— 137.
	PAGE_REFS: [199] 	CURR LEMMA: Becker
HAS LEMMA: Becker 07% afiftentie, afgeweftn. 199,
LEMMA: Becker 07% afiftentie
14 98 0 start_stop 	      Becker 07% afiftentie, a

16 159 7 continue 	         dikant fem :magh  hebben  in  het  verkie-
17 159 2 continue 	         mn  van  cen  Armmeefter 3: te examineeren.
	PAGE_REFS: [688] 	CURR LEMMA: Bleskensgrave
18 163 6 continue_stop 	         688.
19 122 -34 start 	       —__   DrûfJard gelaft ordre te fellen tot bet
20 162 5 continue 	         opvangen  van  [Heydens  en  Vagabonden.
	PAGE_REFS: [692] 	CURR LEMMA: Bleskensgrave
21 163 6 continue_stop 	         692.
22 163 6 continue 	   —— Regenten van de Hage wegens [teke-
23 159 2 continue 	         ve Capitalen, de Raadt van Staate te ad-
	PAGE_REFS: [717] 	CURR LEMMA: Bleskensgrave
24 163 2 continue_stop 	         vijeeren.  717.
25 166 5 continue 	   —— Schouten ende Regenten    mitsgaders
HAS LEMMA: Predikant en Kerkenraadt van de Hage ge-
26 157 -5 start 	         Predikant en Kerkenraadt van de Hage ge-
27 161 0 start 	         laft haar te reguleeren na bet een en twin-
28 161 0 start 	         tighfle Articul van het Reglement op de po-
	PAGE_REF

61 0 -2 start 	 ee


 scan-10-odd-3
	IS INDEX HEADER: 0 201 	## )           E           Xx.
possibly misrecognised repeat symbol: 217 Cs
possibly misrecognised repeat symbol: 131 et
DEVIATING LINE: 240 [118, 117, 110, 108, 240, 117, 116, 235, 238] gj geadmitteert te werden van hare
avg_repeat_symbol_length: 174
DEVIATING LINE: 235 [108, 240, 117, 116, 235, 238, 116, 240, 113] geadmitteert. 491.
avg_repeat_symbol_length: 174
DEVIATING LINE: 238 [240, 117, 116, 235, 238, 116, 240, 113, 235] Pafport om eenige Goederen na Prank-
avg_repeat_symbol_length: 174
DEVIATING LINE: 240 [116, 235, 238, 116, 240, 113, 235, 113, 113] gm eenigh Silverwerck 24 Parijs te
avg_repeat_symbol_length: 174
DEVIATING LINE: 235 [238, 116, 240, 113, 235, 113, 113, 66, 65] item voor de Meubilen en Bágagie
avg_repeat_symbol_length: 174
DEVIATING LINE: 233 [112, 114, 62, 105, 233, 109, 109, 61, 102] Pafport om met vier Paarden de Ri=
avg_repeat_symbol_length: 174
DEVIATING LINE: 233 [109, 109, 61, 102, 233, 110, 21

...

	PAGE_REFS: [70] 	CURR LEMMA: Tarouca
HAS LEMMA: Temet om /ubfiftentie afgeweefen. 70.
LEMMA: Temet om /ubfiftentie afgeweefen
52 53 -24 start_stop 	    Temet  om  /ubfiftentie    afgeweefen.   70.
53 102 22 continue 	      Dad                  a
HAS LEMMA: Tengnagel 7e advifeeren op de Rcgue[le van
54 48 -29 start 	   Tengnagel 7e advifeeren  op de Rcgue[le van
55 96 15 continue 	      Pelgrom Engelbreght om approbatie van [te-
	PAGE_REFS: [713] 	CURR LEMMA: Tengnagel
56 93 15 continue_stop 	      kere Collatie.   713.


 scan-41-even-1
	IS INDEX HEADER: 0 255 	##              !           <
possibly misrecognised repeat symbol: 142 nn
DEVIATING LINE: 227 [56, 103, 54, 227, 103, 105, 54, 98] Pa/port ‘om de Monteeringe voor het
avg_repeat_symbol_length: 144
DEVIATING LINE: 105 [103, 54, 227, 103, 105, 54, 98, 4, 56] te mogen uytvoeren. 451.
avg_repeat_symbol_length: 144
DEVIATING LINE: 108 [54, 98, 4, 56, 108, 53, 92, 225, 101] 192.
avg_repeat_symbol_length: 144
DEVIATING LINE: 225 [56,

26 86 -33 start 	     ———— Refòlutie wegens de Aften:  van des
	PAGE_REFS: [205] 	CURR LEMMA: Vreede
27 134 20 continue_stop 	        charge.   205.
28 94 -22 start 	      —— antwoordt op baar Hoogh Mog. Re:
29 125 9 continue 	       Solutie ‚ raakende de defeftueusheyt, te exa-
	PAGE_REFS: [228] 	CURR LEMMA: Vreede
30 133 23 continue_stop 	        mineeren.   228.
31 85 -31 start 	     —_— propofitie wegens de vyf dertigh duy-
32 123 8 continue 	       Jent guldens van de Admiralitejt   te exa
	PAGE_REFS: [246, 415] 	CURR LEMMA: Vreede
33 132 17 continue_stop 	        mineeren.   246. 415.
34 86 -30 start 	     ——_—  om patent  voor  de  Compagnie vak
35 136 19 continue 	        Haarfma   de Raadt van Staate te advi-
	PAGE_REFS: [263] 	CURR LEMMA: Vreede
36 127 6 continue_stop 	       Jesren.   263.
37 91 -31 start 	      —— gertet op het projet Reglement op de
38 139 21 continue 	        Revijien van fententieh van den Ra4dt van
	PAGE_REFS: [265] 	CURR LEMMA: Vreede
39 141 22 continu

54 121 27 continue 	       ken.   696.                         #5
55 27 -69 start 	  ——  pegfoeck 07: dedommagement van brand
	PAGE_REFS: [791] 	CURR LEMMA: Wierfma
56 124 27 continue_stop 	        afgeweefen.   791.
HAS LEMMA: Windilgratz affcheydt gewalidiceert en Brie-
57 78 -25 start 	     Windilgratz affcheydt  gewalidiceert en  Brie-
58 127 26 continue 	       ven van recredenlie.   EIT.
59 84 -14 start 	     —— Secretaris Medaille van drie honder:
	PAGE_REFS: [626] 	CURR LEMMA: Windilgratz
60 123 13 continue_stop 	       geldens toegelenhbt.  626.
61 127 20 continue 	       wiiee                    Wins


 scan-44-even


 scan-44-even-0
	IS INDEX HEADER: 0 243 	##                        I           N
possibly misrecognised repeat symbol: 223 eeN
possibly misrecognised repeat symbol: 131 A
DEVIATING LINE: 271 [95, 144, 271, 96, 149, 267, 145] oyftigh guldens toegeleght. 267.
avg_repeat_symbol_length: 135
DEVIATING LINE: 267 [144, 271, 96, 149, 267, 145, 268, 142, 142] Refolntie v

 scan-107-even
skipping non-index page


 scan-107-odd
skipping non-index page


 scan-108-even
skipping non-index page


 scan-108-odd
skipping non-index page


 scan-109-even
skipping non-index page


 scan-109-odd
skipping non-index page


 scan-110-even
skipping non-index page


 scan-110-odd
skipping non-index page


 scan-111-even
skipping non-index page


 scan-111-odd
skipping non-index page


 scan-112-even
skipping non-index page


 scan-112-odd
skipping non-index page


 scan-113-even
skipping non-index page


 scan-113-odd
skipping non-index page


 scan-114-even
skipping non-index page


 scan-114-odd
skipping non-index page


 scan-115-even
skipping non-index page


 scan-115-odd
skipping non-index page


 scan-116-even
skipping non-index page


 scan-116-odd
skipping non-index page


 scan-117-even
skipping non-index page


 scan-117-odd
skipping non-index page


 scan-118-even
skipping non-index page


 scan-118-odd
skipping non-index page


 scan-119-even
skipping non-

ZeroDivisionError: division by zero

In [ ]:
for lemma in lemma_index:
    print("\nTrefwoord:", lemma)
    #print(lemma_index[lemma])
    for entry in lemma_index[lemma]:
        pages = ", ".join([str(page_ref) for page_ref in entry["page_refs"]])
        description = entry["description"][:70]
        print("\tPagina:", pages, "\tBeschrijving:", description)

In [ ]:
# scan 45 uneven is first resolution page
# page num: 91

from fuzzy_context_searcher import FuzzyContextSearcher
import pandas as pd

config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 3,
    "skip_size": 0,
}

fuzzy_searcher = FuzzyContextSearcher(config)

keywords = [
    "Admiraliteyt tot Amfterdam", 
    "Admiraliteyt in het Noorder Quartier", 
    "Admiraliteyt in Vrieslandt", 
    "Admiralteyt in Zeelandt",
    "Varckens"
]

distractor_terms = {
    "Admiraliteyt tot Amfterdam": {
        "Admiraliteyt in het Noorder Quartier", "Admiraliteyt in Vrieslandt", "Admiralteyt in Zeelandt"
    },
    "Admiraliteyt in het Noorder Quartier": {
        "Admiraliteyt tot Amfterdam", "Admiraliteyt in Vrieslandt", "Admiralteyt in Zeelandt"
    },
    "Admiraliteyt in Vrieslandt": {
        "Admiraliteyt tot Amfterdam", "Admiraliteyt in het Noorder Quartier", "Admiralteyt in Zeelandt"
    },
    "Admiralteyt in Zeelandt": {
        "Admiraliteyt tot Amfterdam", "Admiraliteyt in het Noorder Quartier", "Admiraliteyt in Vrieslandt"
    },
}
fuzzy_searcher.index_keywords(keywords)
fuzzy_searcher.index_distractor_terms(distractor_terms)

hocr_resolution_pages = []



### Fuzzy Searching of Keywords in the Resolutions

Knowing which keywords should appear in the text, possibly with some spelling variation and OCR errors, we can use a fuzzy search algorithm to find candidate matches. 

Keywords that are similar to each other are registered as distractor terms, so matches are assigned as candidates to the nearest of sets of similar keywords. 

In [ ]:
from parse_republic_hocr_files import merge_text_lines, read_hocr_scan

lemma_matches = defaultdict(list)

def add_context(match, page_text):
    context = fuzzy_searcher.get_term_context(page_text, match, context_size=40)
    match["match_term_in_context"] = context["match_term_in_context"]
    match["context_start_offset"] = context["start_offset"]
    match["context_end_offset"] = context["end_offset"]

for scan_file in scan_files:
    resolution_page_num = scan_file["scan_page_num"] - 90
    if scan_file["scan_page_num"] <= 90:
        continue
    print(scan_file["scan_page_num"], resolution_page_num)
    hocr_page = read_hocr_scan(scan_file)
    page_text = merge_text_lines(hocr_page)
    matches = fuzzy_searcher.find_candidates(page_text)
    for match in matches:
        lemma_matches[match["match_keyword"]] += [match]
        add_context(match, page_text)
        match["page_num"] = scan_file["scan_page_num"]
        print(match["match_keyword"], "\t", match)
    #break
    

In [ ]:
for lemma in sorted(lemma_matches):
    print("\n", lemma, "\tAantal kandidaten:", len(lemma_matches[lemma]), "\n")
    for match in lemma_matches[lemma]:
        print("\tKandidaat:", match["match_string"])
        print("\tPagina:", match["page_num"])
        print("\tContext:", match["match_term_in_context"][5:-5])
        print()



### Extracting Resolutions From Pages

Identify:

- resolution dates
- resolution participant lists
- resolution text blocks

In [533]:
from fuzzy_context_searcher import FuzzyContextSearcher
from fuzzy_person_name_searcher import FuzzyPersonNameSearcher

config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}

keywords = [
    #"Is ter Vergaderinge gelefen",
    #"Ontvangen een miffive van",
    #"WAAR op gedelibereert zijnde",
    #"WAAR op geen refolutie is gevallen",
    #"De Resolutien, gifteren genomen",
    #"Nihil actum eft.",
    #"PRAESIDE",
    "PRAESENTIBUS",
    #"Januarii"
]

week_days = [
    "Lunae",
    "Martis",
    "Mercurii",
    "Jovis",
    "Veneris",
    "Sabbathi",
    "Dominica"
]

months = [
    "Januarii",
    "Februarii",
    "Maart",
    "April",
    "Mey",
    "Junii",
    "Juli",
    "Augufti",
    "September",
    "October",
    "November",
    "December"
]

spelling_variants = {
    "Ontvangen een miffive van": [
        "ON een miffive van",
    ],
    "PRAESENTIBUS": [
        "PRASENTIBUS",
        "PRESENTIBUS"
    ]
}

fuzzy_searcher = FuzzyContextSearcher(config)
fuzzy_person_searcher = FuzzyPersonNameSearcher(config)

fuzzy_searcher.index_keywords(keywords)
#fuzzy_searcher.index_spelling_variants(spelling_variants)
#fuzzy_searcher.index_distractor_terms(distractor_terms)

def is_empty_line(line):
    return len(line["words"]) == 0

def get_resolution_page_paragraphs(page_info):
    paragraphs = []
    for column_info in page_doc["columns"]:
        paragraph = []
        prev_line_bottom = None
        for line in column_info["column_hocr"]["lines"]:
            boundary = False
            if is_empty_line(line):
                continue
            if prev_line_bottom == None:
                prev_line_bottom = line["bottom"]
                paragraph.append(line)
                continue
            line_gap = line["top"] - prev_line_bottom
            if line_gap > 30:
                boundary = True
            elif line_gap > 10 and line_is_centered_date(line):
                boundary = True
            if boundary and prev_line_bottom < 400:
                paragraph = [] # start new paragraph, ignore header line
            elif boundary:
                paragraphs.append(paragraph)
                paragraph = []
            paragraph.append(line)
            prev_line_bottom = line["bottom"]
        if len(paragraph) > 0:
            paragraphs.append(paragraph)
    return paragraphs

def line_has_weekday(line):
    return line_has_word_from_list(line, week_days)

def line_has_month(line):
    return line_has_word_from_list(line, months)

def line_has_word_from_list(line, word_list):
    for line_word in line["words"]:
        best_match = None
        best_score = 1
        for list_word in word_list:
            score = score_levenshtein_distance(line_word["word_text"], list_word)
            relative_score = score / len(list_word)
            if relative_score < 0.4:
                if relative_score < best_score:
                    best_match = list_word
                    best_score = relative_score
        if best_match:
            print("#{}# #{}#".format(line_word["word_text"], best_match), best_score)
            return True
    return False

def line_is_centered_date(line):
    # line is centered
    if line["words"][0]["left"] < 200 or line["words"][-1]["right"] > 800:
        return False
    if line_has_weekday(line) and line_has_month(line):
        print("\n\tCentered date line:\t", line["line_text"])
        return True
    # line has weekday den number month
    return False

def paragraph_starts_with_centered_date(paragraph):
    if line_is_centered_date(paragraph[0]):
        return True
    return False

def merge_paragraph_lines(paragraph):
    paragraph_text = ""
    for line in paragraph:
        if line["line_text"][-1] == "-":
            paragraph_text += line["line_text"][:-1]
        else:
            paragraph_text += line["line_text"] + " "
    return paragraph_text

for page_id in pages_info:
    if pages_info[page_id]["page_type"] != "resolution_page":
        continue
    if pages_info[page_id]["scan_num"] < 264 or  pages_info[page_id]["scan_num"] > 280:
        continue
    print(page_id, pages_info[page_id]["page_type"])
    page_doc = retrieve_page_doc(page_id)
    for paragraph in get_resolution_page_paragraphs(page_doc):
        paragraph_text = merge_paragraph_lines(paragraph)
        matches = fuzzy_searcher.find_candidates(paragraph_text, include_variants=True)
        if len(matches) == 0 and paragraph_starts_with_centered_date(paragraph):
            print("DATE LINE:", paragraph_text)
        #    print("paragraph_text:", paragraph_text)
        for match in matches:
            #print("\t", match)
            if match["match_term"] == "PRAESENTIBUS":
                print("DAY START:", paragraph_text)
            context_match = fuzzy_searcher.get_term_context(paragraph_text, match, context_size=200)
            #print(context_match)
            person_matches = fuzzy_person_searcher.find_person_names_in_text(context_match["match_term_in_context"])
            #person_matches = fuzzy_person_searcher.find_person_names_in_context(context_match)
            #for person_match in person_matches:
            #    print("\t", person_match)
        #print("\n\n\n")


scan-264-even resolution_page
scan-264-odd resolution_page
#Martis# #Martis# 0.0
#Junit# #Junii# 0.2

	Centered date line:	 Martis den 19, Junit
DAY START: Martis den 19, Junit 1725. PRASIDE, Den Heere Van Maasdam. PRASENTIBUS. De Heeren Jan Welderen van Heucketom , Singendonck , ván Heeckeren, Umibgroeven met een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt. Vanden Boetzelaar Raadtpenfionaris van Hoornbeeck. Ocker(Je. Van Voor. Van Schwartzenbergty Rou[e , Vriefen. Emmen, Tamminga. 
scan-265-even resolution_page
scan-265-odd resolution_page
scan-266-even resolution_page
scan-266-odd resolution_page
DAY START: Mercuri: den 210. Junii 4725. PRASTDE, Den Heere Zan Maasdam. PRASENTIBUS, De Heeren’ Van Welderen van Heuckts lom van Heeckeren Umbgroeven met een extraordinaris Gedeputeerde uyt dé Provincie van Gelderlandt. Panden Boetzelaar Eelbo van Marfeveen, Boon Raadtpen/ionaris van Hoornbeeck. OckerJe, met een extraordinaris Gedeputeerde uyt de Provincie van Zeelandt. Van

## Experimental Corner

Playground for trying fuzzy searching algorithms.

In [503]:
def score_levenshtein_distance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1
    distances = range(len(s1) + 1)
    #distances = [0] * (len(s1) + 1)
    #print("distances initial:",distances)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        #print("distances_:", distances_)
        #print("i2:", i2, "c2:", c2)
        for i1, c1 in enumerate(s1):
            #print("i1:", i1, "c1:", c1)
            if c1 == c2:
                distances_.append(distances[i1])
                #print(distances_, "equal")
            else:
                #print("\tminimum of:", distances[i1], distances[i1 + 1], distances_[-1])
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
                #print(distances_, "unequal")
        distances = distances_
        #distances = distances_
        #print("distances:", distances, s2[i2:i2+len(s1)])
    return distances[-1]

def get_match(matrix, T):
    match = ""
    col_index = len(matrix) - 1
    distance = matrix[-1][-1]
    row_index = len(matrix[-1]) - 1
    while row_index > 0:
        if col_index < 0:
            print("BREAKING")
            break
        #print("col_index:", col_index, "row_index:", row_index, "distance:", distance)
        if matrix[col_index-1][row_index-1] < distance:
            #print("replacing")
            match = T[col_index] + match
            distance = matrix[col_index-1][row_index-1]
            col_index -= 1
            row_index -= 1
        elif matrix[col_index][row_index-1] < distance:
            #print("inserting")
            #match = T[col_index-1] + match
            distance = matrix[col_index][row_index-1]
            row_index -= 1
        elif matrix[col_index-1][row_index] < distance:
            #print("deleting")
            match = T[col_index] + match
            distance = matrix[col_index-1][row_index]
            col_index -= 1
        elif matrix[col_index-1][row_index-1] == distance:
            #print("copying")
            match = T[col_index] + match
            col_index -= 1
            row_index -= 1
        else:
            print("This should never be printed")
        #print("row_index:", row_index)
        #print("match:", match)
    #print("returning match:", match)
    return (match, col_index+1, matrix[-1][-1])

def search_levenshtein_distance(P, T, threshold=2):
    if len(P) > len(T):
        P, T = T, P
    distances = range(len(P) + 1)
    matches = []
    matrix = []
    #distances = [0] * (len(T) + 1)
    #print("distances initial:",distances)
    for t_j, T_j in enumerate(T):
        distances_ = [0]
        #print("distances_:", distances_)
        #print("t_j:", t_j, "T_j:", T_j)
        for p_i, P_i in enumerate(P):
            #print("p_i:", p_i, "P_i:", P_i)
            if P_i == T_j:
                distances_.append(distances[p_i])
                #print(distances_, "equal")
            else:
                #print("\tminimum of:", distances[p_i], distances[p_i + 1], distances_[-1])
                distances_.append(1 + min((distances[p_i], distances[p_i + 1], distances_[-1])))
                #print(distances_, "unequal")
        distances = distances_
        #distances = distances_
        #print("distances:", distances, T[t_j-len(P):t_j])
        matrix += [distances]
        distance = distances[-1]
        if distance <= threshold:
            match = get_match(matrix, T)
            #print(T[:match[1]])
            #print(T[match[1]:match[1]+len(match[0])])
            #print(T[match[1]+len(match[0]):])
            matches.append(match)
    #for row in matrix:
    #    print(row)
    #for col_index, distance in enumerate(distances):
    return matches

target = "survey"
context = "surgery"
matches = search_levenshtein_distance(target, context)
print(matches)

target = "survey"
context = "the doctor is specialised in surgery of the heart"
matches = search_levenshtein_distance(target, context)
print(matches)



[('surge', 0, 2), ('surger', 0, 2), ('surgery', 0, 2)]
[('surge', 29, 2), ('surger', 29, 2), ('surgery', 29, 2)]


In [327]:
texts = [
    "B. dr Bofch Mikelaer, zal op den 11 May , in de Keyzers Kroon verkopen , een uvrmuntënde pirty Schilderyen van de voornaernfte M;efter«; ils van de Oude Pslma extra goed , A'exander Veronecs, Pluweele Breugel , Wouwerman, J. vander Heydc, Laireffe, Bril en ancVc. Nagelaten door een voon.aem Liefhcbb.T ; als mede een fraye parry Teekenmgen en Miniatuur fchoone D'ukke Prenten van de voorn-emfte Mcefters. De Citilogu-zal ir.tyds te bekomen zyn by Jicób Carpi Konft Schilder, en by de gsm. Mikelaer. B. SÜgtenborff Mikelaer, za' op Woensdag den 25- Miy, t'Amft. in 't Oude Heeren Logement verkopen, een pirty Engelfche Manuft-Uuren et Wmkclw ren , beftaende in Lakend diverfe fbrteering vin * en 9 quart breed , Biyen, Kirfayen, Drogetreu,.Sergies, gcfc»om-c en gj'lrcepre Kalamankcn, S.urynen, diverfe Grynen en Stoften, Ssycn, Chitzen , Catoenenen Neteldoekeu , divetfc Kouffen,-1 !___. witte gebleekte. Li.inens„cn andere Goederen meer ;, alle* dacgj ïoer <k Verkoping te zien.. '",
    "Plülippus van d:r Land Makelaer, za! op d:.i : 5 May verkopen , een uytmuntend ko1 fti- Kabinet Schilderyen; als vin Pb. Woaverman, A. vin O.tilc, Rottenhimcr, G. Metzu, G. dc LairefTc, D. van Deeien, |. Srccn, D. Teniers, de- Oude Griffier. ]. en 4. Bcth, M. ds Hond-koere-, J. do Heem, J. Lingelbigh.en andere Mecfterj meer; nagelaten door den £J. Heer Secretaris Lambert Witzen , wier van de Citalogus by den gen*. Mikelaer te bekomen zullen zyn. Alte de gee-.e die iers te prereideeren hebben of verfftiulcKgt zyn op de mgchten Boedel vin Pieter Engels , tot Warmer overleden, gelieven luc prerenficn binr.e.-yden tyd vaa 5 weeleen 1:1 te leveren leo Wccsvi-lers rn Armvoogden tot Wormer ; dew-'ke de Penningen vm de y;em. Boedel geprovenieert binnen korre , na verloop van de ge.eyde C weken , by preferentie en concurrentie zulten diftri- MSfctti foi.nig als zy zu'ten vmdc.i te be'nojren,",
    "Jac. Torner Junior Mikelaer, prefènteert tegens primo May.of wel eerder :e verbuurcn, een zeer vermakelyke en welgelege Herberg genaemt RUSTENBURG ; gelegen buyten dc Urregtfj Poort op 't Ruften!., rger par! , met zyn tnodicufe Huyzinge , fraye Thuyn en Tuynhuy», Kolfoaen.Troktafel &o; aiwaer de neering veel jiren met fueces il gcoaen en nog werd gecontinueert, laetft bewoont dexr wylen Dirk He Lange. N ider onderngtinge bv gem. Makelae-. Alle de eeene die eenig regt, aélie of pretenfie beeft, of iets fchal lig zyn a-.n den r.eabindonneerden Boedel .-an Dirk van Schaik en Ja -metje Theuniffe, geweezene Herbe-g er in de Oude Loosdregt, werden-verzoet zulks op en aen ce geven ter Secretary aldaer, uyterlyk voor primo April 1745 , op potne vau eeuwig ftilfwygen en verftek.",
    "ArnoHns van Sprang-Mikelaer, zal op Maendag d-n 27 Vnny.t'Amfteid. in 't Oude Heeten Logement vetkopen, een wel ter nerinc 4_endc Hoys, In de Crom elboo»ftee» 't rte huvs -an den\"Dim,»rnaemt het Mo huys, daer de Baftille uythangt ; breeJet byßiljettr Vermeld : lemant nadet ondetrigi begreiende of genegen zynde dit Perceel uyt de hand te-koopen , fpieeke met de gem. Makelaet, by van eygendom «dagen voet de Veikoopdag te zien tullen zyn. Ajnüld:. s- au Spiang.M.kc!aei, zal op Maendag den 2- lünv , t'Aml. in 't Ó.de tfeeien Logement vetku. en, drie hegre, fterkeen weC-doortt üsnefde Huvfcn, ftaende nieft mi'k.nrlrien in deGroore aen dl-Vtoord-vde by de dwirsftiact • bieede by 3i,;**rte**i vermeld: lefnanrna-e.'onderri.ti.ig begeerende of geneegen z.ynie degemPi'tcedriaTt ie.__.ad.teko.pen,fpi?eke mei yucru, Mak_liei, by wien de bewyzen .van cyi*j_d-_- S dagen voot de:yeikoop_ag.tc _iea zullen ü\", cv",
    "Job. Haverkamp Mikelaer, zal op Maendag den 11 January 1745, t Amft. in t Oude Heere Logement verkopen, No. 1. Een hegt en fterk Huys; ftaende op de Weesperftraet tuffchen de Hect:n en Keyzersgragt. No.1. Eén dito Huys.ftaende naeft 't voorgaende. No. 3. Een Sdepart in een we! ter nering ftaende Huys en Erve op de N. Z. van de Agterburgwal op de hoek van de Pottebakkersfteeg. Breder by Biüetten gefpecifkeert. De bewyzen van eygendom en de Vylconditien zullen 8 dagen voor en op de Verkoopdag voormiddag» tot J 2 uuren toe te zien zyn ten Compt. van den Notaris E. Haverkamp. Uyt dé huni te Koop een wel beklante Banket en Confiturier» Winkel met deszelfs Gereetfchappen , waer in die affaire» circa SS jaeren met c.\".cd fucces zyn gecontinueert: Te bevragen by Hendrik Bofch en Gomp., in de Kalverftract, tuflehen de Öjjes-flaysen Olyfiigers-fteeg\" t'Amfterdam. Als meede het Huys te hu*r.",
    "J. van Zutphen Mikelaer , zal op Maendag den 14 September , t'Amft. in 't Oude Heeren Logement verkopen , een extra wel geleegen 3-ftreeksLiken-Raem , met zyn Thuyn , Huyzinge en Droogfcheevders Winkel, eertyts genaemt het Varken, en nu Buyren-Ruft, ftaende en aeleegen buyten de Raem-Pooit ren eynde het eerfte Sceene-Pad , zynde Stads Grond , en geteekent met. No! 27, 28 en 29, Alles breder by Biljetten en dagelyks te zien , de bewyzen van Eygendom zullen 8 dagen voor en op de verkoopdag te zien zyn '. ten Compt. van den Nots. Arnoud Roermond in de Krom-Elleboogfteeg , en nader onderrigting by gem. Makelaer. ; Alle die iets te pretendeeren mogten hebben ten.laften van wylen Jean Fredrik Bernard , in zyn Leven Boekverkoper t'Amft. v of die eenige Engagementen met dezelve hebben lopende , uyr wat hoofde het o:)-_zoude mogen zyn, als mede die aen der Boedel fchuJdig mogten wezen , of ook eenige Goederen van hem onder zig hebben , werden verzogt zulks op voor tieu ïsOitcber 1744. .;ten Compt. vanden Notaris Jan Ardinois , op de Cingel t'Amfterdam.",    
]

target = "Makelaar"
target = "Oude Heere Logement"
for text in texts:
    matches = search_levenshtein_distance(target, text)
    print(matches)



[('Oude Heeren Logemen', 568, 2), ('Oude Heeren Logement', 568, 1), ('Oude Heeren Logement ', 568, 2)]
[]
[]
[('Oude Heeten Logement', 73, 2)]
[('Oude Heere Logeme', 74, 2), ('Oude Heere Logemen', 74, 1), ('Oude Heere Logement', 74, 0), ('Oude Heere Logement ', 74, 1), ('Oude Heere Logement v', 74, 2)]
[('Oude Heeren Logemen', 74, 2), ('Oude Heeren Logement', 74, 1), ('Oude Heeren Logement ', 74, 2)]
